# 1. 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [2]:
import pickle

In [115]:
service = Service('chromedriver.exe')
driver = webdriver.Chrome(service = service)

In [4]:
isbn13 = pd.read_pickle('isbn13.pkl')

In [6]:
isbn13

0        9791190710343
1        9791190710350
2        9791190710367
3        9791128858338
4        9788954685603
             ...      
54971    9791185020136
54972    9788997838127
54973    9788956591858
54974    9788987794938
54975    9788995696972
Name: isbn13, Length: 54976, dtype: object

In [ ]:
rank_mean = [] # 도서 평균 평점
rank_percentage = [] # 평균 percentage
review_title = [] # 리뷰 제목
review_id = [] # 리뷰 id
review_rank = [] # 리뷰 평점
review_text = [] # 리뷰 
isbn_idx = []


for isbn in isbn13[2024:3000]: # 페이지 입력
    url = "http://www.yes24.com/product/search?query={}".format(isbn)
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 페이지에 있는 도서 링크(50개)
    book_links = soup.select('a.gd_name')
    goods = []
    for link in book_links:
        a= link['href'].split('/')[3] 
        goods.append(a)
    for item in goods:
        del(goods[1:])
        driver.get('http://www.yes24.com'+'/Product/Goods/{}'.format(item)) # 상품 페이지
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        try:
            mean = soup.select('div.gd_rating > em.yes_b')[0].text
            rank_mean.append([mean]) # 도서 평균 평점
        except:
            mean = []
            rank_mean.append(mean) # 도서 평균 평점
        rank_p = []
        try:
            for k in range(5):
                rank_ = soup.select('span.barGraphBase')[k].text # 평점 percentage
                rank_p.append(rank_)
            rank_percentage.append(rank_p)
        except:
            rank_ = []
            rank_percentage.append(rank_)
        try:
            review_t = []  # 각각의 책의 리뷰제목 리스트
            review_i = []
            review_r = []
            review_te = []
            for page in range(1,3): # 리뷰 페이지 2개
                driver.get('http://www.yes24.com/Product/communityModules/GoodsReviewList/{}?Type=ALL&_=1648001707174&Sort=2&PageNumber={}'.format(item, page))
                time.sleep(1)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                
                blog_link = []
                for i in range(5): # 리뷰 한 페이지에 있는 5개 리뷰 가져오기
                    try:
                        title = soup.select('span.review_tit')[i].text.strip() # 리뷰 제목
                        review_t.append(title)
                        r_id = soup.select('a.lnk_id')[i].text # 리뷰 id
                        review_i.append(r_id)
                        rank = soup.select('span.review_rating')[i].text.split('\n')[1][-2:] # 리뷰 평점
                        review_r.append(rank)
                        blog_l = soup.select('div.review_lnk a')[i]['href'] # 리뷰 블로그 링크
                        blog_link.append(blog_l)
                    except:
                        pass
                for j in range(5): # 리뷰 한 페이지에 있는 5개 블로그 내용 가져오기
                        driver.get(blog_link[j]) # 리뷰 블로그 링크 들어가기
                        time.sleep(1)
                        html = driver.page_source
                        soup = BeautifulSoup(html, 'html.parser')
                        text = soup.select('div.blogContArea')[0].text # 리뷰 내용 
                        review_te.append(text)
        except:
            pass
        review_title.append(review_t)
        review_id.append(review_i)
        review_rank.append(review_r)
        review_text.append(review_te)
        isbn_idx.append(isbn)

In [109]:
col = ['isbn','rank_mean','rank_percentage','review_title','review_id','review_rank','review_text']
user = pd.DataFrame([isbn_idx,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text]).T
user.columns = col
user

,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text
0,[9.6],"[85%, 15%, 0%, 0%, 0%]","[행복해지려는 관성, 내 식대로의 행복 찾기, [서평] 행복해지려는 관성 - 행복을...","[책****곰, 자*련, f*****o, 2****a, 박*리, n***e, p**...","[5점, 3점, 4점, 5점, 5점, 5점, 5점, 5점, 5점, 4점]","[\n \n그러므로 이제는 안다. 좋아하는 것은 결코 잘하는 것과 같지 않으며, 돈..."
1,[8.5],"[50%, 50%, 0%, 0%, 0%]","[연애와 술, 결혼 이후 아내는 여간해서는 더이상 술에 입을 대지 않는다... 김괜...","[목*화, k******i]","[5점, 3점]",[\n연애와 술 『시간의 흐름』 시리즈 책들의 소재가 마음에 와 닿았다. 나를...
2,[],[],[],[],[],[]
3,[9.6],"[86%, 14%, 0%, 0%, 0%]","[풋코는 폭스테리어, 노견일기 3, 풋코, 어느새 3편이 나왔습니다., 노견 보호자...","[n***8, s****t, 잔*, 원*갱, h******5, 슈*, p***1]","[4점, 5점, 5점, 5점, 5점, 5점, 5점]",[\n 개 종류는 잘 모르지만 폭스테리어는 좀 사납다는 말 들은 듯하다. 물지 않고...
4,[9.6],"[90%, 10%, 0%, 0%, 0%]","[[모지스 할머니의 크리스마스 선물], Happy Christmas; 모지스 할머니...","[크****이, 해**이, 인*캣, h***n, 크*벨, 2****a, g*****...","[5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 4점, 4점]",[\n 제목이 눈에 들어와 고르게 된 책이다. 크리스마스 분위기를 이전처럼\n느끼기...
...,...,...,...,...,...,...
219,[9.4],"[67%, 33%, 0%, 0%, 0%]","[배려, 다정한 시선으로 바라보는 일부터, 배려, 어렵고도 쉬운 배려라는 말에 대해...","[자*련, 나*이, 신*****리, j******2, t******e, m*****...","[4점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점]",[\n배려가 무엇인지 알아야 잘 할 수 있다. 상황을 이해하고 타인을 생각하고 나 ...
220,[7.6],"[17%, 50%, 33%, 0%, 0%]","[그저 쓱~ 룰루~, 얼큰한것이, 낮의 목욕탕과 술, 낮의 목욕탕과 술, 목욕탕]","[K*l, 썬*, e*******e, m******6, c********2]","[4점, 5점, 5점, 4점, 4점]","[\n혼자서도 밥 잘먹고, 혼자서도 영화 잘보는데, 정말 못하겠는건 혼술이다. 물론..."
221,[],[],[],[],[],[]
222,[9.3],"[71%, 29%, 0%, 0%, 0%]","[[꽃은 젖어도 향기는 젖지 않는다] 도종환, 이것이 시인의 삶이다, 세월의 고통이...","[디***스, 무*, 낙*, 모*루, 상***래, 책**께, 뿌****무]","[4점, 4점, 5점, 5점, 5점, 5점, 5점]","[\n고 노무현 대통령의 노제 때 사회를 맡아, 하늘을 향해 ""노무현 당신을 사랑합..."


In [110]:
user.to_pickle('user_2000-2023', protocol = 4)

In [112]:
user1 = pd.read_pickle('user_2000-2023') # 수정 전